## imports

In [ ]:
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from econml.dr import DRLearner
from lightgbm import LGBMClassifier
from sklift.models import SoloModel

from sklift.viz import plot_qini_curve
import os
import sys
from pathlib import Path
import yaml
from datetime import datetime
from sklift.metrics import qini_auc_score
import seaborn as sns

import pandas as pd
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt


In [ ]:
cwd = Path.cwd()
repo_root = ([cwd] + list(cwd.parents))[1]

# Ensure repo_root is on sys.path so `src.train` can be imported
sys.path.append(str(repo_root))
from src.process_datasets import create_data, get_web_feats

# Load the YAML config file
with open(os.path.join(repo_root, "config.yaml"), "r") as f:
    config = yaml.safe_load(f)
timestamp = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")

In [ ]:
from src.data_handler import DataHandler
from src.trainers import compare_train_test_performance, calculate_qini_auuc
from src.data_process import check_column_consistency,check_missing_values,dataset_health_check,check_column_consistency,check_missing_values,analyze_data_drift,dataset_health_check

## data handling

### data loading

In [ ]:
# ----------------------------
# Load data
# ----------------------------
app_usage = pd.read_csv(repo_root / "data" / "train" / "app_usage.csv")
web_visits = pd.read_csv(repo_root / "data" / "train" / "web_visits.csv")
claims = pd.read_csv(repo_root / "data" / "train" / "claims.csv")
churn_labels = pd.read_csv(repo_root / "data" / "train" / "churn_labels.csv")
test_app_usage = pd.read_csv(repo_root / "data" / "test" / "test_app_usage.csv")
test_web_visits = pd.read_csv(repo_root / "data" / "test" / "test_web_visits.csv")
test_claims = pd.read_csv(repo_root / "data" / "test" / "test_claims.csv")
test_churn_labels = pd.read_csv(repo_root / "data" / "test" / "test_churn_labels.csv")

### data preprocess

In [ ]:
train_data_handler = DataHandler(day_first_web=True)
test_data_handler = DataHandler(day_first_web=False)
X_train, y_train, treatment_train = train_data_handler.get_data(
    app_usage, web_visits, claims, churn_labels
)
X_test, y_test, treatment_test = test_data_handler.get_data(test_app_usage, test_web_visits, test_claims, test_churn_labels)



In [ ]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)
y_test = y_test.fillna(0)
treatment_train = treatment_train.fillna(0)
treatment_test = treatment_test.fillna(0)

In [ ]:


# --- Execution ---
dataset_health_check(X_train, X_test)

print("\n=========================================")
print("          HEALTH CHECK COMPLETE          ")
print("=========================================")

### feature selection

In [ ]:


# Combine X_train with churn and outreach
X_train_with_labels = X_train.copy()
X_train_with_labels["churn"] = y_train
X_train_with_labels["outreach"] = treatment_train

# Compute the correlation matrix
corr_matrix = X_train_with_labels.corr()

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=False, cmap="coolwarm", fmt=".2f", vmin=-1, vmax=1)
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
# Compute the correlation matrix
corr_matrix = X_train_with_labels.corr()

# Filter the correlation matrix to show only 'churn' and 'outreach' correlations with all features
filtered_corr = corr_matrix.loc[["churn", "outreach"]]

# Display the filtered correlation matrix
plt.figure(figsize=(12, 2))
sns.heatmap(filtered_corr, annot=True, cmap="coolwarm", fmt=".2f", vmin=-1, vmax=1)
plt.title("Correlation of Churn and Outreach with Features")
plt.show()

In [ ]:
## Correlation Analysis: Identify Redundant Features

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Get correlation matrix (already computed in previous cell)
corr_abs = corr_matrix.abs()

# 1. Find highly correlated feature pairs (threshold: 0.8)
upper_triangle = np.triu(np.ones(corr_abs.shape), k=1).astype(bool)
high_corr_pairs = []

for i in range(len(corr_abs.columns)):
    for j in range(i + 1, len(corr_abs.columns)):
        if corr_abs.iloc[i, j] > 0.8:
            high_corr_pairs.append(
                {
                    "Feature_1": corr_abs.columns[i],
                    "Feature_2": corr_abs.columns[j],
                    "Correlation": corr_abs.iloc[i, j],
                }
            )

high_corr_df = pd.DataFrame(high_corr_pairs).sort_values("Correlation", ascending=False)

print("=" * 80)
print("HIGHLY CORRELATED FEATURES (|r| > 0.8)")
print("=" * 80)
if len(high_corr_df) > 0:
    print(high_corr_df.head(20).to_string(index=False))
    print(f"\nTotal pairs: {len(high_corr_df)}")
else:
    print("✅ No feature pairs with |correlation| > 0.8")

# 2. Check correlation with target (churn) and treatment (outreach)
target_corr = (
    corr_matrix[["churn", "outreach"]].abs().sort_values("churn", ascending=False)
)

print("\n" + "=" * 80)
print("FEATURE CORRELATION WITH CHURN & OUTREACH")
print("=" * 80)
print(target_corr.head(20).to_string())

# 3. Identify features to potentially drop
# Rule: If two features correlated > 0.8, drop the one with lower correlation to churn
features_to_drop = set()

for _, row in high_corr_df.iterrows():
    feat1, feat2 = row["Feature_1"], row["Feature_2"]

    # Skip if already marked for dropping
    if feat1 in features_to_drop or feat2 in features_to_drop:
        continue

    # Compare correlation with churn
    if feat1 in target_corr.index and feat2 in target_corr.index:
        corr1 = target_corr.loc[feat1, "churn"]
        corr2 = target_corr.loc[feat2, "churn"]

        # Drop the feature with weaker correlation to churn
        if corr1 < corr2:
            features_to_drop.add(feat1)
        else:
            features_to_drop.add(feat2)

print("\n" + "=" * 80)
print("RECOMMENDED FEATURES TO DROP (Redundant + Weak Churn Correlation)")
print("=" * 80)
if len(features_to_drop) > 0:
    print(f"Total features to drop: {len(features_to_drop)}")
    print(sorted(features_to_drop))
else:
    print("✅ No redundant features detected")

# 4. Variance Inflation Factor (VIF) check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Select only numeric features (exclude target/treatment if still present)
numeric_cols = [
    col
    for col in X_train.columns
    if col not in ["churn", "outreach"] and X_train[col].dtype in ["float64", "int64"]
]

# Sample if too many features (VIF is slow)
if len(numeric_cols) > 50:
    print(
        f"\n⚠️  Sampling 50 features for VIF analysis (full dataset has {len(numeric_cols)} features)"
    )
    numeric_cols_sample = np.random.choice(numeric_cols, 50, replace=False).tolist()
else:
    numeric_cols_sample = numeric_cols

# Calculate VIF
vif_data = pd.DataFrame()
vif_data["Feature"] = numeric_cols_sample
vif_data["VIF"] = [
    variance_inflation_factor(X_train[numeric_cols_sample].fillna(0).values, i)
    for i in range(len(numeric_cols_sample))
]
vif_data = vif_data.sort_values("VIF", ascending=False)

print("\n" + "=" * 80)
print("VARIANCE INFLATION FACTOR (VIF) - Top 20 Features")
print("VIF > 10 indicates severe multicollinearity")
print("=" * 80)
print(vif_data.head(20).to_string(index=False))

# 5. PCA Recommendation
print("\n" + "=" * 80)
print("PCA vs FEATURE SELECTION RECOMMENDATION")
print("=" * 80)

n_features = X_train.shape[1]
n_high_corr = len(high_corr_df)
n_vif_high = (vif_data["VIF"] > 10).sum()

print(f"\n📊 DATASET SUMMARY:")
print(f"   Total features: {n_features}")
print(f"   Highly correlated pairs: {n_high_corr}")
print(f"   Features with VIF > 10: {n_vif_high}")
print(f"   Recommended to drop: {len(features_to_drop)}")

print(f"\n💡 RECOMMENDATION:")

if n_high_corr > 20 or n_vif_high > 10:
    print("   ⚠️  HIGH MULTICOLLINEARITY DETECTED")
    print("   → Option 1: Drop redundant features (simple, interpretable)")
    print(
        "   → Option 2: Use PCA (loses interpretability, may help with regularization)"
    )
    print("\n   For WellCo uplift modeling (per data/wellco_client_brief.txt):")
    print("   ✅ RECOMMENDED: Drop redundant features")
    print("      - Preserves clinical interpretability (ICD-10 codes, claims_count)")
    print("      - LightGBM handles remaining collinearity well")
    print("   ❌ AVOID PCA for now:")
    print("      - Loses domain meaning (can't explain 'PC1' to clinicians)")
    print("      - Use only if dimensionality reduction is critical for compute")
else:
    print("   ✅ MODERATE MULTICOLLINEARITY")
    print(
        "   → Drop {0} redundant features (correlation > 0.8)".format(
            len(features_to_drop)
        )
    )
    print("   → PCA not needed — LightGBM handles remaining correlations")

# 6. Save features to drop for reproducibility
if len(features_to_drop) > 0:
    output_path = repo_root / "outputs" / f"features_to_drop_{timestamp}.txt"
    with open(output_path, "w") as f:
        f.write("\n".join(sorted(features_to_drop)))
    print(f"\n✅ Redundant features list saved to: {output_path}")

In [ ]:
## Drop Redundant Features (If Recommended Above)

if len(features_to_drop) > 0:
    print(f"Dropping {len(features_to_drop)} redundant features...")
    X_train_reduced = X_train.drop(columns=list(features_to_drop))
    X_test_reduced = X_test.drop(columns=list(features_to_drop))

    print(f"Original shape: {X_train.shape}")
    print(f"Reduced shape:  {X_train_reduced.shape}")

    # Use X_train_reduced for modeling
else:
    print("No features to drop — proceed with current feature set")
    X_train_reduced = X_train.copy()
    X_test_reduced = X_test.copy()

In [ ]:
train_data_outreach_grouped = X_train_reduced.groupby(treatment_train).mean()
train_data_outreach_grouped

In [ ]:
train_data_outreach_grouped = X_train.groupby(treatment_train).mean()
train_data_outreach_grouped

In [ ]:
## Analyze Heterogeneous Treatment Effects by Segment


# Prepare data with labels
X_train_with_labels = X_train.copy()
X_train_with_labels["churn"] = y_train.values
X_train_with_labels["outreach"] = treatment_train.values

# Calculate overall ATE for reference
overall_ate = (
    y_train[treatment_train == 1].mean() - y_train[treatment_train == 0].mean()
)

# Features to analyze (WellCo clinical priorities)
features_ate = X_train.columns.tolist()

for feature in features_ate:
    if feature not in X_train.columns:
        print(f"\n⚠️  Feature '{feature}' not found in X_train, skipping...")
        continue

    # Check if feature has variation
    if X_train[feature].nunique() <= 1:
        print(f"\n⚠️  Feature '{feature}' has no variation, skipping...")
        continue

    # Handle features with few unique values (binary/categorical)
    unique_vals = X_train[feature].nunique()

    try:
        if unique_vals <= 3:
            # For binary/low-cardinality features, use actual values as segments
            X_train_with_labels[f"{feature}_level"] = X_train[feature].astype(str)
            segments = sorted(X_train_with_labels[f"{feature}_level"].unique())
        else:
            # For continuous features, create tertiles
            X_train_with_labels[f"{feature}_level"] = pd.qcut(
                X_train[feature],
                q=3,
                labels=["Low", "Medium", "High"],
                duplicates="drop",  # Handle tied values
            )
            segments = ["Low", "Medium", "High"]

        # Calculate ATE for each segment
        segment_ates = []
        for segment in segments:
            mask = X_train_with_labels[f"{feature}_level"] == segment

            # Check if we have both treated and control members in this segment
            n_treated = (mask & (treatment_train == 1)).sum()
            n_control = (mask & (treatment_train == 0)).sum()

            if n_treated == 0 or n_control == 0:
                print(
                    f"\n⚠️  Segment '{segment}' has no treated or control members, skipping..."
                )
                continue

            treated_churn = X_train_with_labels.loc[
                mask & (treatment_train == 1), "churn"
            ].mean()
            control_churn = X_train_with_labels.loc[
                mask & (treatment_train == 0), "churn"
            ].mean()

            segment_ate = treated_churn - control_churn
            n_members = mask.sum()

            segment_ates.append(
                {
                    "Segment": f"{segment} {feature}",
                    "ATE": segment_ate,
                    "Members": n_members,
                    "N_Treated": n_treated,
                    "N_Control": n_control,
                    "Treated_Churn": treated_churn,
                    "Control_Churn": control_churn,
                }
            )

        if len(segment_ates) == 0:
            print(f"\n⚠️  No valid segments for '{feature}', skipping...")
            continue

        ate_df = pd.DataFrame(segment_ates)

        # Print results
        print("\n" + "=" * 80)
        print(f"TREATMENT EFFECT BY {feature.upper()} LEVEL")
        print("=" * 80)
        print(ate_df.to_string(index=False))
        print(f"\nOverall ATE (reference): {overall_ate:.4f}")
        print("=" * 80)

        # Visualize
        fig, ax = plt.subplots(figsize=(12, 6))
        x = range(len(ate_df))
        colors = ["#d62728" if ate > 0 else "#2ca02c" for ate in ate_df["ATE"]]

        bars = ax.bar(x, ate_df["ATE"], color=colors, alpha=0.7, edgecolor="black")

        # Add value labels on bars
        for i, (idx, row) in enumerate(ate_df.iterrows()):
            ax.text(
                i,
                row["ATE"],
                f'{row["ATE"]:.3f}',
                ha="center",
                va="bottom" if row["ATE"] > 0 else "top",
                fontsize=9,
            )

        # Reference lines
        ax.axhline(y=0, color="black", linestyle="-", linewidth=1, alpha=0.3)
        ax.axhline(
            y=overall_ate,
            color="blue",
            linestyle="--",
            linewidth=2,
            label=f"Overall ATE ({overall_ate:.4f})",
        )

        ax.set_xticks(x)
        ax.set_xticklabels(ate_df["Segment"], rotation=45, ha="right")
        ax.set_ylabel("Treatment Effect (Treated Churn - Control Churn)", fontsize=11)
        ax.set_title(
            f"Heterogeneous Treatment Effects by {feature}\n(Negative = Outreach Reduces Churn)",
            fontsize=12,
            fontweight="bold",
        )
        ax.legend(loc="best")
        ax.grid(axis="y", alpha=0.3)

        plt.tight_layout()
        plt.show()

        # Clinical interpretation for WellCo
        print(f"\n💡 CLINICAL INSIGHT for {feature}:")
        best_segment = ate_df.loc[ate_df["ATE"].idxmin()]
        worst_segment = ate_df.loc[ate_df["ATE"].idxmax()]

        if best_segment["ATE"] < overall_ate:
            print(
                f"   ✅ BEST: {best_segment['Segment']} shows strongest benefit (ATE={best_segment['ATE']:.4f})"
            )
            print(
                f"      → Prioritize outreach for this segment ({best_segment['Members']} members)"
            )

        if worst_segment["ATE"] > 0:
            print(
                f"   ⚠️  WORST: {worst_segment['Segment']} shows harm/no benefit (ATE={worst_segment['ATE']:.4f})"
            )
            print(
                f"      → Avoid outreach for this segment ({worst_segment['Members']} members)"
            )

    except Exception as e:
        print(f"\n❌ Error processing '{feature}': {str(e)}")
        continue

print("\n" + "=" * 80)
print(
    "SUMMARY: Use these insights to target outreach based on uplift model predictions"
)
print("=" * 80)

## training

### maunal t learn

In [ ]:


# ---------------------------
#  dataset
# ---------------------------
X = X_train
treatment = treatment_train
y = y_train
# ---------------------------
# 1. Propensity score estimation
# ---------------------------
prop_model = LogisticRegression(max_iter=1000)
prop_model.fit(X, treatment)
propensity_scores = prop_model.predict_proba(X)[:, 1]

# ---------------------------
# 2. Train separate outcome models (T-learner)
# ---------------------------
weights_treated = 1 / propensity_scores
weights_control = 1 / (1 - propensity_scores)

model_treated = RandomForestClassifier(n_estimators=100, random_state=42)
model_treated.fit(
    X[treatment == 1], y[treatment == 1], sample_weight=weights_treated[treatment == 1]
)

model_control = RandomForestClassifier(n_estimators=100, random_state=42)
model_control.fit(
    X[treatment == 0], y[treatment == 0], sample_weight=weights_control[treatment == 0]
)

# ---------------------------
# 3. Predict outcomes under treatment and control
# ---------------------------
y_pred_treated = model_treated.predict_proba(X_test)[:, 1]
y_pred_control = model_control.predict_proba(X_test)[:, 1]

uplift_scores = y_pred_treated - y_pred_control


# ---------------------------
# 4. Qini curve
# ---------------------------
def qini_curve(y, treatment, uplift):
    """Compute Qini curve points"""
    df = pd.DataFrame({"y": y, "t": treatment, "uplift": uplift})
    df = df.sort_values("uplift", ascending=False)

    cum_treated = df["t"].cumsum()
    cum_control = (~df["t"].astype(bool)).cumsum()

    cum_outcome_treated = (df["y"] * df["t"]).cumsum()
    cum_outcome_control = (df["y"] * (1 - df["t"])).cumsum()

    # Qini: difference in cumulative outcomes scaled by group sizes
    qini = cum_outcome_treated - cum_outcome_control * (
        cum_treated / np.maximum(cum_control, 1)
    )
    return qini


qini = qini_curve(y_test, treatment_test, uplift_scores)


# Qini curve subplot
plt.subplot(1, 2, 1)
plt.plot(np.arange(1, len(qini) + 1), qini, label="Uplift model")
plt.plot([0, len(qini)], [0, qini.max()], "--", label="Random model")
plt.xlabel("Number of individuals targeted")
plt.ylabel("Cumulative incremental response")
plt.title("Qini Curve")
plt.legend()

# compute Qini AUC (area between model qini curve and random model)
x = np.arange(1, len(qini) + 1)
qini_vals = qini.values

model_auc = np.trapz(qini_vals, x)
random_line = np.linspace(0, qini_vals.max(), len(qini_vals))
random_auc = np.trapz(random_line, x)

qini_auc = model_auc - random_auc
qini_auc_norm = qini_auc / random_auc if random_auc != 0 else np.nan

print(f"Qini AUC (area over random): {qini_auc:.4f}")
print(f"Normalized Qini AUC: {qini_auc_norm:.4f}")

# annotate qini plot
plt.subplot(1, 2, 1)
plt.annotate(
    f"Qini AUC: {qini_auc:.3f}\nNorm: {qini_auc_norm:.3f}",
    xy=(0.6 * len(qini_vals), 0.6 * qini_vals.max()),
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", alpha=0.8),
)

# second subplot: uplift score distribution for diagnostics
plt.subplot(1, 2, 2)
plt.hist(uplift_scores, bins=30, color="C1", edgecolor="k")
plt.xlabel("Uplift score (treated - control)")
plt.ylabel("Count")
plt.title("Uplift score distribution")
plt.tight_layout()
plt.show()

### s learner

In [ ]:
## Elastic Net Logistic Regression

logistic_elasticnet = LogisticRegression(
    random_state=42,
    max_iter=5000,
    class_weight="balanced",
    penalty="elasticnet",  # Mix of L1 and L2
    solver="saga",
    C=0.2,
    l1_ratio=0.1,  # 0.5 = equal mix of L1 and L2
)

slearner_logistic_enet = SoloModel(estimator=logistic_elasticnet)
slearner_logistic_enet.fit(X_train, y_train, treatment=treatment_train)

uplift_test_enet = slearner_logistic_enet.predict(X_test)
test_qini_enet = qini_auc_score(y_test, uplift_test_enet, treatment_test)
print(f"Elastic Net Logistic Test Qini AUC: {test_qini_enet:.4f}")
uplift_train_enet = slearner_logistic_enet.predict(X_train)
train_qini_enet = qini_auc_score(y_train, uplift_train_enet, treatment_train)
print(f"Elastic Net Logistic Train Qini AUC: {train_qini_enet:.4f}")

In [ ]:
compare_train_test_performance(
    slearner_logistic_enet,
    X_train,
    X_test,
    y_train,
    y_test,
    treatment_train,
    treatment_test,
    model_name="Baseline S-Learner",
)

In [ ]:
## Uplift Score Variance Analysis

print("\n" + "=" * 80)
print("UPLIFT SCORE VARIANCE DIAGNOSTICS")
print("=" * 80)

print(f"\nUplift score range:")
print(f"  Min: {uplift_test_enet.min():.6f}")
print(f"  Max: {uplift_test_enet.max():.6f}")
print(f"  Range: {uplift_test_enet.max() - uplift_test_enet.min():.6f}")
print(f"  Mean: {uplift_test_enet.mean():.6f}")
print(f"  Std: {uplift_test_enet.std():.6f}")

print(
    f"\n⚠️  ISSUE: Very narrow uplift range ({uplift_test_enet.max() - uplift_test_enet.min():.6f})"
)
print(f"   This indicates weak heterogeneous treatment effects")
print(f"   Model predicts similar uplift for most members")

# Visualize distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
ax1.hist(uplift_test_enet, bins=50, edgecolor="black", alpha=0.7)
ax1.axvline(
    uplift_test_enet.mean(),
    color="red",
    linestyle="--",
    linewidth=2,
    label=f"Mean: {uplift_test_enet.mean():.6f}",
)
ax1.set_xlabel("Uplift Score (Predicted Treatment Effect)")
ax1.set_ylabel("Frequency")
ax1.set_title("Distribution of Uplift Scores (Test Set)", fontweight="bold")
ax1.legend()
ax1.grid(alpha=0.3)

# Box plot with percentiles
ax2.boxplot(uplift_test_enet, vert=True)
ax2.set_ylabel("Uplift Score")
ax2.set_title("Uplift Score Distribution (Box Plot)", fontweight="bold")
ax2.grid(alpha=0.3, axis="y")

# Add percentile annotations
percentiles = [10, 25, 50, 75, 90]
for p in percentiles:
    val = np.percentile(uplift_test_enet, p)
    ax2.axhline(val, color="gray", linestyle=":", alpha=0.5)
    ax2.text(1.15, val, f"P{p}: {val:.5f}", fontsize=8)

plt.tight_layout()
plt.savefig(
    repo_root / "outputs" / f"uplift_distribution_{timestamp}.png",
    dpi=150,
    bbox_inches="tight",
)
plt.show()

print(f"\n✅ Distribution plot saved to: outputs/uplift_distribution_{timestamp}.png")
print("=" * 80)

### DRLearner

In [ ]:


# ----------------------------------------------------------------------
# 1. Data Loading and Preparation
df_test = pd.DataFrame(
    {
        "churn": y_test,
        "outreach": treatment_test,
        "member_id": np.arange(10000),  # Placeholder member IDs
    }
)
# -------------------------------------------------

# ----------------------------------------------------------------------
# 2. EconML Model Training (DRLearner) - Corrected
# ----------------------------------------------------------------------

# Define the models
model_propensity = LogisticRegression(solver="lbfgs")  # P(T|X)
# IMPORTANT: Added model_regression for E[Y|X, T], using Regressor for continuous Y
model_regression = RandomForestRegressor(
    n_estimators=100, max_depth=3, min_samples_leaf=10, random_state=42
)
model_final = GradientBoostingRegressor(
    n_estimators=100, max_depth=3, min_samples_leaf=10, random_state=42
)  # Final CATE Estimator (tau(X))

# DRLearner model instance - Using correct parameters: model_propensity and model_regression
dr_learner = DRLearner(
    model_final=model_final,
    model_propensity=model_propensity,  # Corrected from 'model_t'
    model_regression=model_regression,  # Added for complete doubly robust estimation
    cv=5,
    mc_iters=3,
    random_state=42,
)

print("Training DRLearner on training set...")
# Fit the model using the training data
dr_learner.fit(
    Y=y_train,
    T=treatment_train,
    X=X_train,
    W=None,  # Assuming no features W that only affect the outcome, not the treatment effect
)

# ----------------------------------------------------------------------
# 3. CATE Prediction (Uplift Score) - Using Test Data
# ----------------------------------------------------------------------

# Predict CATE on the held-out TEST set
uplift_test_dr = dr_learner.effect(X_test)
test_qini_dr = qini_auc_score(y_test, uplift_test_dr, treatment_test)
uplift_train_dr = dr_learner.effect(X_train)
train_qini_dr = qini_auc_score(y_train, uplift_train_dr, treatment_train)


# Add results to the test DataFrame
df_results_test = df_test.copy()
df_results_test["prioritization_score"] = uplift_test_dr

# Sort and Rank the test members
df_results_test = df_results_test.sort_values(
    by="prioritization_score", ascending=True
).reset_index(drop=True)
df_results_test["rank"] = df_results_test.index + 1

# Output the required ranked list
top_n_members_test = df_results_test[["member_id", "prioritization_score", "rank"]]
print("\nTop 5 Ranked Members (Test Set):")
print(top_n_members_test.head())
top_n_members_test.to_csv("ranked_members_for_outreach_test.csv", index=False)

# ----------------------------------------------------------------------
# 4. Qini/AUUC Calculation and Plotting (Using Test Data)
# ----------------------------------------------------------------------



# Calculate and plot the results
df_qini_test, auuc_score_test, total_uplift_gain_test = calculate_qini_auuc(
    df_results_test
)

# Plotting the Qini Curve
plt.figure(figsize=(10, 6))
plt.plot(
    df_qini_test["k"], df_qini_test["qini_gain"], label="Model Qini Curve (Uplift)"
)
plt.plot(
    df_qini_test["k"],
    df_qini_test["random_gain"],
    linestyle="--",
    color="gray",
    label="Random Targeting Baseline",
)

plt.title(f"Qini Curve (Test Set) - AUUC: {auuc_score_test:.4f}")
plt.xlabel("Number of Targeted Members (k)")
plt.ylabel("Cumulative Uplift (Net Churn Reduction)")
plt.legend()
plt.grid(True)
plt.savefig("qini_curve_test.png")
print(f"\n--- Uplift Model Metrics (Test Set) ---")
print(f"Total Theoretical Uplift Gain: {total_uplift_gain_test:.4f}")
print(f"Area Under the Uplift Curve (AUUC, Normalized): {auuc_score_test:.6f}")
print("--- Uplift Model Metrics (Test Set) ---")

In [ ]:
## Uplift Score Variance Analysis

print("\n" + "=" * 80)
print("UPLIFT SCORE VARIANCE DIAGNOSTICS")
print("=" * 80)

print(f"\nUplift score range:")
print(f"  Min: {uplift_test_dr.min():.6f}")
print(f"  Max: {uplift_test_dr.max():.6f}")
print(f"  Range: {uplift_test_dr.max() - uplift_test_dr.min():.6f}")
print(f"  Mean: {uplift_test_dr.mean():.6f}")
print(f"  Std: {uplift_test_dr.std():.6f}")

print(f"   This indicates weak heterogeneous treatment effects")
print(f"   Model predicts similar uplift for most members")

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
plot_qini_curve(
    y_train,
    uplift_train_dr,
    treatment_train,
    perfect=True,
    name="DR-Learner Train",
    ax=ax1,
)
ax1.set_title(f"DR-Learner - Train (Qini AUC: {train_qini_dr:.4f})")
plot_qini_curve(
    y_test,
    uplift_test_dr,
    treatment_test,
    perfect=True,
    name="DR-Learner Test",
    ax=ax2,
)
ax2.set_title(f"DR-Learner - Test (Qini AUC: {test_qini_dr:.4f})")
plt.tight_layout()
plt.show()

### t learner

In [ ]:
from sklift.models import TwoModels

# Create SEPARATE instances for treatment and control to avoid sklearn error
lgbm_t_treatment = LGBMClassifier(
    random_state=42,
    n_jobs=-1,
    n_estimators=100,  # Reduced from 300
    max_depth=3,  # Shallow trees to prevent overfitting
    learning_rate=0.1,  # Increased from 0.03
    min_child_samples=100,  # Increased from 30
    reg_alpha=0.5,  # Strong L1 regularization
    reg_lambda=0.5,  # Strong L2 regularization
    min_split_gain=0.1,  # Require minimum gain to split
    subsample=0.7,  # Bagging
    colsample_bytree=0.7,  # Feature sampling
)

lgbm_t_control = LGBMClassifier(
    random_state=43,  # Different seed
    n_jobs=-1,
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    min_child_samples=100,
    reg_alpha=10.0,
    reg_lambda=5.0,
    min_split_gain=0.1,
    subsample=0.7,
    colsample_bytree=0.7,
)

tlearner = TwoModels(
    estimator_trmnt=lgbm_t_treatment, estimator_ctrl=lgbm_t_control, method="vanilla"
)

tlearner.fit(X_train, y_train, treatment=treatment_train)

# Evaluate
uplift_train_tlearner = tlearner.predict(X_train)
uplift_test_tlearner = tlearner.predict(X_test)

train_qini_tlearner = qini_auc_score(y_train, uplift_train_tlearner, treatment_train)
test_qini_tlearner = qini_auc_score(y_test, uplift_test_tlearner, treatment_test)

print("\n" + "=" * 70)
print("T-LEARNER (REGULARIZED) WITH LIGHTGBM")
print("=" * 70)
print(f"Train Qini AUC: {train_qini_tlearner:.4f}")
print(f"Test Qini AUC:  {test_qini_tlearner:.4f}")
print(f"Overfitting Gap: {train_qini_tlearner - test_qini_tlearner:.4f}")
print("=" * 70)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
plot_qini_curve(
    y_train,
    uplift_train_tlearner,
    treatment_train,
    perfect=True,
    name="T-Learner Train",
    ax=ax1,
)
ax1.set_title(f"T-Learner - Train (Qini AUC: {train_qini_tlearner:.4f})")
plot_qini_curve(
    y_test,
    uplift_test_tlearner,
    treatment_test,
    perfect=True,
    name="T-Learner Test",
    ax=ax2,
)
ax2.set_title(f"T-Learner - Test (Qini AUC: {test_qini_tlearner:.4f})")
plt.tight_layout()
plt.show()

### X learner

In [ ]:
## Experiment 2: X-Learner Implementation

print("\n" + "=" * 80)
print("X-LEARNER (ADVANCED META-LEARNER)")
print("=" * 80)

print("\nX-Learner steps:")
print("  1. Train models on treatment/control groups (like T-Learner)")
print("  2. Impute counterfactual outcomes")
print("  3. Train models to predict treatment effects directly")
print("  4. Weight by propensity scores")

# Step 1: Train base models on treatment and control
from sklearn.linear_model import Ridge

# Models for stage 1
mu0_model = Ridge(alpha=10.0, random_state=42)  # Control group model
mu1_model = Ridge(alpha=10.0, random_state=43)  # Treatment group model

# Split by treatment
X_train_treated = X_train[treatment_train == 1]
y_train_treated = y_train[treatment_train == 1]
X_train_control = X_train[treatment_train == 0]
y_train_control = y_train[treatment_train == 0]

print(f"\nTraining data split:")
print(f"  Treatment group: {len(X_train_treated)} samples")
print(f"  Control group: {len(X_train_control)} samples")

# Fit stage 1 models
mu0_model.fit(X_train_control, y_train_control)
mu1_model.fit(X_train_treated, y_train_treated)

# Step 2: Impute counterfactuals
# For treated: D_1 = Y_1 - mu_0(X_1)
# For control: D_0 = mu_1(X_0) - Y_0

D1 = y_train_treated.values - mu0_model.predict(X_train_treated)  # Treated ITE
D0 = mu1_model.predict(X_train_control) - y_train_control.values  # Control ITE

print(f"\nImputed treatment effects:")
print(f"  Treated group ITE mean: {D1.mean():.6f}")
print(f"  Control group ITE mean: {D0.mean():.6f}")

# Step 3: Train models to predict treatment effects
tau0_model = Ridge(alpha=10.0, random_state=44)  # ITE model on control
tau1_model = Ridge(alpha=10.0, random_state=45)  # ITE model on treatment

tau0_model.fit(X_train_control, D0)
tau1_model.fit(X_train_treated, D1)

# Step 4: Predict with propensity weighting
# tau(x) = g(x) * tau_0(x) + (1 - g(x)) * tau_1(x)
# where g(x) is propensity score

if "propensity_score" in X_test.columns:
    propensity_test = X_test["propensity_score"].values
else:
    # Estimate propensity if not available
    from sklearn.linear_model import LogisticRegression

    prop_model = LogisticRegression(random_state=42, max_iter=1000)
    prop_model.fit(X_train, treatment_train)
    propensity_test = prop_model.predict_proba(X_test)[:, 1]

# Final X-Learner prediction
tau0_pred = tau0_model.predict(X_test)
tau1_pred = tau1_model.predict(X_test)

uplift_test_xlearner = propensity_test * tau0_pred + (1 - propensity_test) * tau1_pred

# Train set predictions
if "propensity_score" in X_train.columns:
    propensity_train = X_train["propensity_score"].values
else:
    propensity_train = prop_model.predict_proba(X_train)[:, 1]

tau0_pred_train = tau0_model.predict(X_train)
tau1_pred_train = tau1_model.predict(X_train)
uplift_train_xlearner = (
    propensity_train * tau0_pred_train + (1 - propensity_train) * tau1_pred_train
)

# Evaluate
train_qini_xlearner = qini_auc_score(y_train, uplift_train_xlearner, treatment_train)
test_qini_xlearner = qini_auc_score(y_test, uplift_test_xlearner, treatment_test)

print(f"\nX-Learner Performance:")
print(f"  Train Qini AUC: {train_qini_xlearner:.4f}")
print(f"  Test Qini AUC:  {test_qini_xlearner:.4f}")
print(f"  Overfitting Gap: {train_qini_xlearner - test_qini_xlearner:.4f}")
print(f"\nUplift score statistics (test):")
print(f"  Range: {uplift_test_xlearner.max() - uplift_test_xlearner.min():.6f}")
print(f"  Mean: {uplift_test_xlearner.mean():.6f}")
print(f"  Std: {uplift_test_xlearner.std():.6f}")
print("=" * 80)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
plot_qini_curve(
    y_train,
    uplift_train_xlearner,
    treatment_train,
    perfect=True,
    name="X-Learner Train",
    ax=ax1,
)
ax1.set_title(f"X-Learner - Train (Qini AUC: {train_qini_xlearner:.4f})")
plot_qini_curve(
    y_test,
    uplift_test_xlearner,
    treatment_test,
    perfect=True,
    name="X-Learner Test",
    ax=ax2,
)
ax2.set_title(f"X-Learner - Test (Qini AUC: {test_qini_xlearner:.4f})")
plt.tight_layout()
plt.show()

## models comparison

In [ ]:
## Visualize All Meta-Learners (Qini Curves)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

models_viz = [
    ("S-Learner (Baseline)", uplift_test_enet, test_qini_enet),
    ("T-Learner", uplift_test_tlearner, test_qini_tlearner),
    ("X-Learner", uplift_test_xlearner, test_qini_xlearner),
    ("DR-Learner", uplift_test_dr, test_qini_dr),
]


for idx, (name, uplift, qini) in enumerate(models_viz):
    if idx < len(axes):
        plot_qini_curve(
            y_test, uplift, treatment_test, perfect=True, name=name, ax=axes[idx]
        )
        axes[idx].set_title(
            f"{name}\nQini AUC: {qini:.4f}", fontweight="bold", fontsize=10
        )
        axes[idx].grid(alpha=0.3)

# Hide unused subplots
for idx in range(len(models_viz), len(axes)):
    axes[idx].axis("off")

plt.tight_layout()
plt.savefig(
    repo_root / "outputs" / f"meta_learner_qini_comparison_{timestamp}.png",
    dpi=150,
    bbox_inches="tight",
)
plt.show()

print(
    f"✅ Qini comparison saved to: outputs/meta_learner_qini_comparison_{timestamp}.png"
)

In [ ]:
## Uplift Distribution Comparison

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, (name, uplift, _) in enumerate(models_viz):
    if idx < len(axes):
        axes[idx].hist(uplift, bins=50, edgecolor="black", alpha=0.7, color="steelblue")
        axes[idx].axvline(
            uplift.mean(),
            color="red",
            linestyle="--",
            linewidth=2,
            label=f"Mean: {uplift.mean():.5f}",
        )
        axes[idx].set_xlabel("Uplift Score")
        axes[idx].set_ylabel("Frequency")
        axes[idx].set_title(
            f"{name}\nRange: {uplift.max() - uplift.min():.5f}",
            fontweight="bold",
            fontsize=10,
        )
        axes[idx].legend(fontsize=8)
        axes[idx].grid(alpha=0.3, axis="y")

# Hide unused subplots
for idx in range(len(models_viz), len(axes)):
    axes[idx].axis("off")

plt.tight_layout()
plt.savefig(
    repo_root / "outputs" / f"meta_learner_distributions_{timestamp}.png",
    dpi=150,
    bbox_inches="tight",
)
plt.show()

print(
    f"✅ Distribution comparison saved to: outputs/meta_learner_distributions_{timestamp}.png"
)